In [1]:
import SimpleITK as sitk  # For loading the dataset
import pandas as pd
import numpy as np  # For data manipulation
import glob  # For populating the list of files
from scipy.ndimage import zoom  # For resizing
import re  # For parsing the filenames (to know their modality)
import os
import csv

In [2]:
data = pd.read_csv('labels1.csv',index_col=False)
data.index+=1

In [3]:
# Divide by class
df_class_0 = data[data['Label'] == 'GBM']
df_class_1 = data[data['Label'] == 'Recurrent GBM']

In [4]:
arr = []
label =[]
with open('labels1.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        arr.append(row['Image_path'].strip('\ufeff'))
        label.append(row['Label'])

In [5]:
for i in range(len(arr)):
    s1, s2 = os.path.split(arr[i])
    xx = pd.DataFrame({'Label':label[i], 'Image_path1':s1, 'Image_path2':s2}, index=[0]) 
    xx.to_csv('Gangs.csv', mode = 'a', header=False,index=False)

In [6]:
cs = pd.read_csv("Gangs.csv", index_col=False) 
cs1 = cs['Patient_ID']

In [7]:
unique_patients = cs1.unique()
print(unique_patients)

['Keck_0010' 'Keck_0011' 'Keck_0020' 'Keck_0023' 'Keck_0028' 'Keck_0032'
 'Keck_0036' 'Keck_0037' 'Keck_0042' 'Keck_0051' 'Keck_0058' 'Keck_0059'
 'Keck_0064' 'Keck_0065' 'Keck_0066' 'Keck_0070' 'Keck_0071' 'Keck_0072'
 'Keck_0082' 'Keck_0086' 'Keck_0087' 'Keck_0088' 'Keck_0095' 'Keck_0097'
 'Keck_0099' 'Keck_0102' 'Keck_0104' 'Keck_0107' 'Keck_0111' 'Keck_0114'
 'Keck_0119' 'Keck_0126' 'Keck_0129' 'Keck_0137' 'Keck_0138' 'Keck_0143'
 'Keck_0147' 'Keck_0148' 'Keck_0155' 'Keck_0158' 'Keck_0161' 'Keck_0162'
 'Keck_0164' 'Keck_0170' 'Keck_0173' 'Keck_0174' 'Keck_0184' 'Keck_0187'
 'Keck_0190' 'Keck_0196' 'Keck_0198' 'Keck_0206' 'Keck_0210' 'Keck_0212'
 'Keck_0215' 'Keck_0217' 'Keck_0218' 'Keck_0221' 'Keck_0225' 'Keck_0227'
 'Keck_0230' 'Keck_0232' 'Keck_0239' 'Keck_0242' 'Keck_0246' 'Keck_0248'
 'Keck_0252' 'Keck_0257' 'Keck_0258' 'Keck_0261' 'Keck_0264' 'Keck_0266'
 'Keck_0275' 'Keck_0281' 'Keck_0282' 'Keck_0284' 'Keck_0285' 'Keck_0290'
 'Keck_0295' 'Keck_0297' 'Keck_0298' 'Keck_0301' 'K

In [8]:
Label_unique =[]
count = 0

for j in range(len(unique_patients)):
    count = 0
    for i in range(len(cs['Patient_ID'])):
        if count == 0 :
            if unique_patients[j] == cs['Patient_ID'][i]:
                count = count + 1
                Label_unique.append(cs['Label'][j])

In [9]:
df = pd.DataFrame(Label_unique)

In [10]:
from collections import Counter


print(Counter(Label_unique).keys()) # equals to list(set(words))
print(Counter(Label_unique).values())

dict_keys(['GBM', 'Recurrent GBM'])
dict_values([87, 28])


In [11]:

Reference_csv = pd.DataFrame({'Label':Label_unique, 'Patient_ID':unique_patients})
Reference_csv.to_csv('Reference.csv', index=False)




In [12]:
df_train = pd.read_csv('Reference.csv')
target_count = df_train.Label.value_counts()
print(target_count)
df_class_0 = df_train[df_train['Label'] == 'GBM']
df_class_1 = df_train[df_train['Label'] == 'Recurrent GBM']

GBM              87
Recurrent GBM    28
Name: Label, dtype: int64


In [13]:
print(len(df_train))
print(df_train)

115
             Label Patient_ID
0              GBM  Keck_0010
1              GBM  Keck_0011
2              GBM  Keck_0020
3              GBM  Keck_0023
4              GBM  Keck_0028
..             ...        ...
110            GBM  Keck_0392
111            GBM  Keck_0393
112            GBM  Keck_0394
113            GBM  Keck_0402
114  Recurrent GBM  Keck_0409

[115 rows x 2 columns]


In [14]:
Patient_ID = []
labels =[]
with open('Reference.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        Patient_ID.append(row['Patient_ID'])
        labels.append(row['Label'])

In [29]:
from radiomics.firstorder import RadiomicsFirstOrder
from radiomics.shape import RadiomicsShape
from radiomics.shape2D import RadiomicsShape2D
from radiomics.glrlm import RadiomicsGLRLM
from radiomics.ngtdm import RadiomicsNGTDM
from radiomics.gldm import RadiomicsGLDM
from radiomics.glcm import RadiomicsGLCM
from radiomics.glszm import RadiomicsGLSZM

import six

import pandas as pd
import os


def getFirstOrder(imageName, maskName):
    image = sitk.ReadImage(imageName)
    mask = sitk.ReadImage(maskName)
    glcmFeatures = RadiomicsFirstOrder(image, mask)
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val
    return rows



def getShape(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glcmFeatures = RadiomicsShape(image, mask)
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val
        rows['Compactness1FeatureValue'] = glcmFeatures.getCompactness1FeatureValue()
        rows['Compactness2FeatureValue'] = glcmFeatures.getCompactness2FeatureValue()

    return rows

def getShape2D(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glcmFeatures = RadiomicsShape2D(image, mask, force2D=True)
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val

    return rows

def getNGTDM(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glrlmFeatures = RadiomicsNGTDM(image, mask, )
    glrlmFeatures.enableAllFeatures()
    results = glrlmFeatures.execute()

    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val

    return rows

def getGLRLM(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glrlmFeatures = RadiomicsGLRLM(image, mask, )
    glrlmFeatures.enableAllFeatures()
    results = glrlmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val
    return rows

def getGLDM(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glcmFeatures = RadiomicsGLDM(image, mask, )
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val

    return rows

def getGLCM(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glcmFeatures = RadiomicsGLCM(image, mask, )
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val

    return rows

def getGLSZM(image, mask):
    image = sitk.ReadImage(image)
    mask = sitk.ReadImage(mask)
    glcmFeatures = RadiomicsGLSZM(image, mask, )
    glcmFeatures.enableAllFeatures()
    results = glcmFeatures.execute()
    rows = {}
    for (key, val) in six.iteritems(results):
        rows[key] = val

    return rows

def combine_dict(one, two,three, four, five, six, seven, imageName):
    one['file'] = imageName

    for k, v in two.items():
        if k not in one.keys():
            one[k] = v
        else:
            one["{}_2".format(k)] = v

    for k, v in three.items():
        if k not in one.keys():
            one[k] = v
        else:
            one["{}_3".format(k)] = v

    for k, v in four.items():
        if k not in one.keys():
            one[k] = v
        else:
            one["{}_4".format(k)] = v

    for k, v in five.items():
        if k not in one.keys():
            one[k] = v
        else:
            print(k, '4')
            one["{}_5".format(k)] = v

    for k, v in six.items():
        if k not in one.keys():
            one[k] = v
        else:
            one["{}_6".format(k)] = v

    for k, v in seven.items():
        if k not in one.keys():
            one[k] = v
        else:
            print(k, '6')
            one["{}_7".format(k)] = v

    return one

In [38]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    seg1 =[]
    seg2 =[]
    seg4 =[]

    t1=[]
    t2=[]
    t1ce=[]
    flair=[]
    
    getFirstOrders = []


    seg1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg1.nii.gz'))
    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg2.nii.gz'))
    seg4.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg4.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1.nii.gz'))
    t2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t2.nii.gz'))
    t1ce.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1ce.nii.gz'))
    flair.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*flair.nii.gz'))
    


    number = [len(seg1[0]), len(seg2[0]), len(seg4[0]), len(t1[0]), len(t2[0]), len(t1ce[0]), len(flair[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg1[0][j])

                    two = getShape(t1[0][j], seg1[0][j])

                    three = getNGTDM(t1[0][j], seg1[0][j])

                    four = getGLRLM(t1[0][j], seg1[0][j])

                    five = getGLDM(t1[0][j], seg1[0][j])

                    six_value = getGLCM(t1[0][j], seg1[0][j])

                    seven = getGLSZM(t1[0][j], seg1[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures1.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures1.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [39]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg2.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures2.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures2.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [40]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg4.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures3.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures3.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [41]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg1.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t2.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures4.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures4.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [42]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg2.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t2.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures5.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures5.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [43]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg4.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t2.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures6.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures6.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [44]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg1.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1ce.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures7.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures7.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [45]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg2.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1ce.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures8.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures8.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [46]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg4.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*t1ce.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures9.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures9.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [47]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg1.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*flair.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures10.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures10.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [48]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg2.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*flair.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures11.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures11.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [49]:
from glob import glob
from radiomics import featureextractor
import six, numpy as np
for i in range(len(Patient_ID)):
    print(i)
    
    seg2 =[]

    t1=[]
    
    getFirstOrders = []


    seg2.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/seg4.nii.gz'))

    t1.append(glob('MRI_Data/DataTransfer/'+ Patient_ID[i] + '/**/*flair.nii.gz'))
    


    number = [ len(seg2[0]),len(t1[0])]
    
    try:
        for j in range(max(number)):
            try:
                    one = getFirstOrder(t1[0][j], seg2[0][j])

                    two = getShape(t1[0][j], seg2[0][j])

                    three = getNGTDM(t1[0][j], seg2[0][j])

                    four = getGLRLM(t1[0][j], seg2[0][j])

                    five = getGLDM(t1[0][j], seg2[0][j])

                    six_value = getGLCM(t1[0][j], seg2[0][j])

                    seven = getGLSZM(t1[0][j], seg2[0][j])

                    res = combine_dict(one, two, three, four, five, six_value, seven, t1[0][j])

                    print("Sruj".format(t1[0][j], len(res)-1))

                    getFirstOrders.append(res)

                
            except ValueError:
                print(ValueError)
                continue

            except IndexError:
                print(IndexError)
                continue
        if i == 0:      
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv("getFeatures12.csv",index = False)
            print(dff.shape)
        else:
            dff = pd.DataFrame(getFirstOrders)
            dff.to_csv('getFeatures12.csv', index = False,mode = 'a',header = None)
        
    except ValueError:
            print(ValueError)
            continue
                
                            

0


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


...

GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj
114


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GrayLevelNonUniformity 4
GrayLevelVariance 4
GrayLevelNonUniformity 6
GrayLevelNonUniformityNormalized 6
GrayLevelVariance 6
Sruj


In [50]:
len(one)

110

In [51]:
len(two)

16

In [52]:
len(three)

5

In [54]:
len(four)

16

In [55]:
len(five)

14

In [59]:
len(seven)

16

In [62]:
res

{'10Percentile': array(335.),
 '90Percentile': array(550.),
 'Energy': array(4.68344566e+09),
 'Entropy': array(3.8475545),
 'InterquartileRange': array(103.),
 'Kurtosis': array(4.92240656),
 'Maximum': array(984.),
 'MeanAbsoluteDeviation': array(67.98435854),
 'Mean': array(443.29336891),
 'Median': array(439.),
 'Minimum': array(111.),
 'Range': array(873.),
 'RobustMeanAbsoluteDeviation': array(43.97526687),
 'RootMeanSquared': array(452.61155025),
 'Skewness': array(0.7525313),
 'TotalEnergy': array(4.68344566e+09),
 'Uniformity': array(0.08686175),
 'Variance': array(8348.20450419),
 'file': 'MRI_Data/DataTransfer/Keck_0409/20160607_153414/20160607_153414_flair.nii.gz',
 'Elongation': 0.8847889929997297,
 'Compactness1FeatureValue': 0.007148592765833149,
 'Compactness2FeatureValue': 0.018156969362261452,
 'Flatness': 0.5892133817118606,
 'LeastAxisLength': 34.146371702851305,
 'MajorAxisLength': 57.952471486042555,
 'Maximum2DDiameterColumn': array(62.17716623),
 'Maximum2DDiame

In [63]:
two

{'Elongation': 0.8847889929997297,
 'Compactness1FeatureValue': 0.007148592765833149,
 'Compactness2FeatureValue': 0.018156969362261452,
 'Flatness': 0.5892133817118606,
 'LeastAxisLength': 34.146371702851305,
 'MajorAxisLength': 57.952471486042555,
 'Maximum2DDiameterColumn': array(62.17716623),
 'Maximum2DDiameterRow': array(62.96824597),
 'Maximum2DDiameterSlice': array(64.81512169),
 'Maximum3DDiameter': array(80.32434251),
 'MeshVolume': array(22614.25),
 'MinorAxisLength': 51.27570888798114,
 'Sphericity': array(0.26283374),
 'SurfaceArea': array(14713.78695738),
 'SurfaceVolumeRatio': array(0.65064227),
 'VoxelVolume': 22862.0}